# Writing to docx
This notebook shows some ways of putting the transcribed audio into docx files (word files). This is often a preferable format for the end user. The process relies on the library `python-docx`, which has functionality to read from and write to docx files.

In [ ]:
%pip install python-docx

In [5]:
import os
from docx import Document
import pandas as pd


project_name = "Example 1"
folder_path = os.path.join("Projects", project_name)
if not os.path.exists(folder_path):
    print("Folder does not exist")

## No speaker information

In [3]:
# Make a docx file from the linebreak separated text file

file_name = "default_whisper_20230124_large-v2_linebreak.txt"

with open(os.path.join(folder_path, file_name), "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f.readlines()]

document = Document()

document.add_heading(f"{project_name} - transcription", 0)
#document.add_page_break()

p = document.add_paragraph()

for line in lines:
    run = p.add_run(str(line)+" ")
    run.add_break()
    #font = run.font
    #font.color.rgb = shared.RGBColor(200, 0, 0)

document.save(os.path.join(folder_path, "Transcription.docx"))

## Speaker information present
If we know who's speaking, we can separate the speech from the different speakers. If saved as a `tsv`, we can iterate over the rows to get the information.

In [8]:
# Read the tsv file into a pandas dataframe
df = pd.read_csv(os.path.join(folder_path, "text_whisper_offset_adjusted.tsv"), sep="\t")

print(f"Shape of df: {df.shape}")
df.head()

,start,end,speaker,text
0,0,4000,Annie Lööf,Det är 16 000 äldre som har...
1,4000,7000,Annie Lööf,...bo på särskilda äldreboenden.
2,7000,11000,Annie Lööf,Och varannat besök på en vårdcentral görs hos ...
3,11000,15000,Annie Lööf,Jag tycker det är för mycket storstadsfokus i ...
4,15000,20000,Annie Lööf,För tar vi oss ut på landsbygden så är det ju ...


In [17]:
# Make a docx file from the tsv file with speaker names

document = Document()

document.add_heading(f"{project_name} - transcription", 0)

p = document.add_paragraph()

last_speaker, curr_speaker = "", ""

# Loop through the dataframe and add a new paragraph for each speaker
for row in range(df.shape[0]):
    curr_speaker = df.loc[row, "speaker"]

    # If the speaker is new, add a new paragraph
    if curr_speaker != last_speaker:
        run = p.add_run()
        run.add_break()
        run.add_break()
        p.add_run(str(curr_speaker)).bold = True
        run = p.add_run()
        run.add_break()
        last_speaker = curr_speaker

    # Add the text to the paragraph
    run = p.add_run(str(df.loc[row, "text"])+" ")
    #run.add_break()

# Save the document
document.save(os.path.join(folder_path, "Transcription fixed.docx"))